In [1]:
# run_baseline_evaluation.py
"""
End-to-end baseline evaluation orchestration
Runs complete pipeline: preprocessing → server → load tests → evaluation
"""

import json
import argparse
import os
from pathlib import Path
import logging
import time

from preprocessing import DataPreprocessor
from server import SingleVariantServer
from load_generator import ClosedLoopLoadGenerator
from metrics import MetricsCalculator
from evaluation import HeldOutEvaluator

logging.basicConfig(
    level=logging.INFO,
    
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def load_data(data_dir: str = "data/processed"):
    """Load preprocessed datasets from JSONL files"""
    train_data = []
    val_data = []
    test_data = []
    
    for split_name, split_list in [("train", train_data), ("val", val_data), ("test", test_data)]:
        path = os.path.join(data_dir, f"{split_name}_data.jsonl")
        if os.path.exists(path):
            logger.info(f"Loading {split_name} data from {path}")
            with open(path, 'r') as f:
                for line in f:
                    if line.strip():
                        split_list.append(json.loads(line))
        else:
            logger.warning(f"File not found: {path}")
    
    logger.info(f"Loaded data: train={len(train_data)}, val={len(val_data)}, test={len(test_data)}")
    
    return train_data, val_data, test_data


In [2]:
def main(args):
    """Run end-to-end baseline evaluation pipeline"""
    
    # Create output directory
    os.makedirs(args.output_dir, exist_ok=True)
    
    logger.info("="*80)
    logger.info("END-TO-END BASELINE EVALUATION: MED-ONLY SERVER (8-BIT QUANTIZATION)")
    logger.info("="*80)
    
    # Step 1: Data preprocessing (if needed)
    if args.preprocess:
        logger.info("\n[STEP 0] PREPROCESSING DATASETS")
        logger.info("-"*80)
        
        preprocessor = DataPreprocessor(
            data_dir=args.data_dir,
            output_dir=args.processed_dir
        )
        train, val, test = preprocessor.run_pipeline()
    
    # Step 2: Load preprocessed data
    logger.info("\n[STEP 1] LOADING DATA")
    logger.info("-"*80)
    
    train_data, val_data, test_data = load_data(args.processed_dir)
    
    if not val_data or not test_data:
        logger.error("No validation or test data found!")
        return
    
    # Use subset for faster iteration (optional)
    if args.data_subset > 0:
        val_data = val_data[:args.data_subset]
        test_data = test_data[:args.data_subset]
        logger.info(f"Using subset: val={len(val_data)}, test={len(test_data)}")
    
    # Step 3: Initialize server
    logger.info("\n[STEP 2] INITIALIZING SERVER")
    logger.info("-"*80)
    
    try:
        server = SingleVariantServer(
            model_name=args.model_name,
            variant="med",
            device=args.device,
            dtype=args.dtype
        )
    except Exception as e:
        logger.error(f"Failed to initialize server: {e}")
        return
    
    # Step 4: Run load tests at multiple concurrency levels
    logger.info("\n[STEP 3] RUNNING LOAD TESTS")
    logger.info("-"*80)
    
    load_test_results = {}
    all_metrics_summary = []
    
    for concurrency in args.concurrencies:
        logger.info(f"\n>>> Testing with concurrency={concurrency}")
        
        # Create load generator
        load_gen = ClosedLoopLoadGenerator(
            inference_func=server.generate,
            max_concurrency=concurrency,
            num_requests=args.num_requests,
            data_loader=val_data
        )
        
        # Run load test
        start_time = time.time()
        metrics = load_gen.run()
        load_duration = time.time() - start_time
        
        # Calculate metrics
        calc = MetricsCalculator(metrics)
        test_metrics = calc.compute_all_metrics()
        load_test_results[concurrency] = test_metrics
        
        # Print report
        calc.print_report(
            title=f"LOAD TEST RESULTS (Concurrency {concurrency})"
        )
        
        # Save detailed metrics to JSON
        metrics_file = os.path.join(
            args.output_dir,
            f"metrics_concurrency_{concurrency}.json"
        )
        calc.save_metrics(metrics_file)
        
        # Save individual request logs
        requests_file = os.path.join(
            args.output_dir,
            f"requests_concurrency_{concurrency}.jsonl"
        )
        load_gen.save_metrics(requests_file)
        
        # Summary entry
        summary_entry = {
            "concurrency": concurrency,
            "num_requests": args.num_requests,
            "duration_sec": load_duration,
            "success_rate": test_metrics["summary"]["success_rate"],
            "throughput_tokens_per_sec": test_metrics["summary"]["throughput_tokens_per_sec"],
            "ttft_p99_ms": test_metrics["ttft"]["p99"],
            "tpot_p95_ms": test_metrics["tpot"]["p95"],
            "e2e_p99_ms": test_metrics["e2e_latency"]["p99"],
            "slo_compliance": test_metrics["summary"]["slo_compliance"],
            "slo_violations": test_metrics["summary"]["slo_violations"]
        }
        all_metrics_summary.append(summary_entry)
    
    # Step 5: Evaluate accuracy on held-out test set
    logger.info("\n[STEP 4] EVALUATING ACCURACY")
    logger.info("-"*80)
    
    try:
        evaluator = HeldOutEvaluator(
            model=server,
            data_loader=test_data,
            batch_size=32
        )
        eval_results = evaluator.evaluate()
        
        # Save eval results
        eval_file = os.path.join(args.output_dir, "eval_results.json")
        with open(eval_file, 'w') as f:
            json.dump(eval_results, f, indent=2)
        
        logger.info(f"Saved evaluation results to {eval_file}")
        
    except Exception as e:
        logger.error(f"Evaluation failed: {e}")
        eval_results = {}
    
    # Step 6: Summary report
    logger.info("\n" + "="*80)
    logger.info("FINAL SUMMARY REPORT")
    logger.info("="*80)
    
    print("\nLoad Test Results by Concurrency:")
    print(f"{'Concurrency':<12} {'Throughput':<18} {'TTFT P99':<12} {'TPOT P95':<12} {'E2E P99':<12} {'SLO Compl':<10}")
    print("-" * 80)
    
    for summary in all_metrics_summary:
        print(f"{summary['concurrency']:<12} "
              f"{summary['throughput_tokens_per_sec']:<18.1f} "
              f"{summary['ttft_p99_ms']:<12.1f} "
              f"{summary['tpot_p95_ms']:<12.1f} "
              f"{summary['e2e_p99_ms']:<12.1f} "
              f"{summary['slo_compliance']*100:<10.1f}%")
    
    print("\nAccuracy Results:")
    if eval_results:
        for dataset_type in sorted(eval_results.keys()):
            if dataset_type != "overall":
                result = eval_results[dataset_type]
                print(f"  {dataset_type.upper():<10s}: {result['em']*100:6.2f}% "
                      f"({result['correct_count']}/{result['total_count']})")
        
        overall = eval_results.get("overall", {})
        print(f"  {'OVERALL':<10s}: {overall.get('em', 0)*100:6.2f}% "
              f"({overall.get('correct_count', 0)}/{overall.get('total_count', 0)})")
    else:
        print("  No evaluation results available")
    
    # Save summary
    summary_file = os.path.join(args.output_dir, "summary.json")
    with open(summary_file, 'w') as f:
        json.dump({
            "load_test_summary": all_metrics_summary,
            "eval_results": eval_results,
            "config": {
                "model_name": args.model_name,
                "variant": "med",
                "num_requests": args.num_requests,
                "concurrencies": args.concurrencies,
                "device": args.device
            }
        }, f, indent=2)
    
    logger.info("\n" + "="*80)
    logger.info("BASELINE EVALUATION COMPLETE")
    logger.info(f"Results saved to: {args.output_dir}")
    logger.info("="*80)
    logger.info("\nFiles generated:")
    logger.info(f"  - summary.json (overall summary)")
    logger.info(f"  - eval_results.json (accuracy metrics)")
    for concurrency in args.concurrencies:
        logger.info(f"  - metrics_concurrency_{concurrency}.json")
        logger.info(f"  - requests_concurrency_{concurrency}.jsonl")



In [3]:
import os
import time
import json

# Define your config here (edit directly in the notebook)
config = {
    "output_dir": "./data/processed",
    "data_dir": "./results/baseline_med",
    "processed_dir": "./data/processed",
    "preprocess": True,
    "data_subset": 0,                 # 0 = use all; >0 = use first N
    "model_name": "meta-llama/Llama-2-7b-chat-hf",
    "device": "mps",
    "dtype": "float16",
    "concurrencies": [1,2,4,8,16,32],
    "num_requests": 5000,
}

def run_pipeline():
    """Run end-to-end baseline evaluation pipeline in notebook."""

    # Convenience local vars (to avoid args.)
    output_dir     = config["output_dir"]
    data_dir       = config["data_dir"]
    processed_dir  = config["processed_dir"]
    preprocess     = config["preprocess"]
    data_subset    = config["data_subset"]
    model_name     = config["model_name"]
    device         = config["device"]
    dtype          = config["dtype"]
    concurrencies  = config["concurrencies"]
    num_requests   = config["num_requests"]

    os.makedirs(output_dir, exist_ok=True)

    logger.info("="*80)
    logger.info("END-TO-END BASELINE EVALUATION: MED-ONLY SERVER (8-BIT QUANTIZATION)")
    logger.info("="*80)

    # Step 1: Data preprocessing (if needed)
    if preprocess:
        logger.info("\n[STEP 0] PREPROCESSING DATASETS")
        logger.info("-"*80)

        preprocessor = DataPreprocessor(
            data_dir=data_dir,
            output_dir=processed_dir
        )
        train, val, test = preprocessor.run_pipeline()

    # Step 2: Load preprocessed data
    logger.info("\n[STEP 1] LOADING DATA")
    logger.info("-"*80)

    train_data, val_data, test_data = load_data(processed_dir)

    if not val_data or not test_data:
        logger.error("No validation or test data found!")
        return

    # Use subset for faster iteration (optional)
    if data_subset > 0:
        val_data = val_data[:data_subset]
        test_data = test_data[:data_subset]
        logger.info(f"Using subset: val={len(val_data)}, test={len(test_data)}")

    # Step 3: Initialize server
    logger.info("\n[STEP 2] INITIALIZING SERVER")
    logger.info("-"*80)

    try:
        server = SingleVariantServer(
            model_name=model_name,
            variant="med",
            device=device,
            dtype=dtype
        )
    except Exception as e:
        logger.error(f"Failed to initialize server: {e}")
        return

    # Step 4: Run load tests at multiple concurrency levels
    logger.info("\n[STEP 3] RUNNING LOAD TESTS")
    logger.info("-"*80)

    load_test_results = {}
    all_metrics_summary = []

    for concurrency in concurrencies:
        logger.info(f"\n>>> Testing with concurrency={concurrency}")

        # Create load generator
        load_gen = ClosedLoopLoadGenerator(
            inference_func=server.generate,
            max_concurrency=concurrency,
            num_requests=num_requests,
            data_loader=val_data
        )

        # Run load test
        start_time = time.time()
        metrics = load_gen.run()
        load_duration = time.time() - start_time

        # Calculate metrics
        calc = MetricsCalculator(metrics)
        test_metrics = calc.compute_all_metrics()
        load_test_results[concurrency] = test_metrics

        # Print report
        calc.print_report(
            title=f"LOAD TEST RESULTS (Concurrency {concurrency})"
        )

        # Save detailed metrics to JSON
        metrics_file = os.path.join(
            output_dir,
            f"metrics_concurrency_{concurrency}.json"
        )
        calc.save_metrics(metrics_file)

        # Save individual request logs
        requests_file = os.path.join(
            output_dir,
            f"requests_concurrency_{concurrency}.jsonl"
        )
        load_gen.save_metrics(requests_file)

        # Summary entry
        summary_entry = {
            "concurrency": concurrency,
            "num_requests": num_requests,
            "duration_sec": load_duration,
            "success_rate": test_metrics["summary"]["success_rate"],
            "throughput_tokens_per_sec": test_metrics["summary"]["throughput_tokens_per_sec"],
            "ttft_p99_ms": test_metrics["ttft"]["p99"],
            "tpot_p95_ms": test_metrics["tpot"]["p95"],
            "e2e_p99_ms": test_metrics["e2e_latency"]["p99"],
            "slo_compliance": test_metrics["summary"]["slo_compliance"],
            "slo_violations": test_metrics["summary"]["slo_violations"]
        }
        all_metrics_summary.append(summary_entry)

    # Step 5: Evaluate accuracy on held-out test set
    logger.info("\n[STEP 4] EVALUATING ACCURACY")
    logger.info("-"*80)

    try:
        evaluator = HeldOutEvaluator(
            model=server,
            data_loader=test_data,
            batch_size=32
        )
        eval_results = evaluator.evaluate()

        # Save eval results
        eval_file = os.path.join(output_dir, "eval_results.json")
        with open(eval_file, 'w') as f:
            json.dump(eval_results, f, indent=2)

        logger.info(f"Saved evaluation results to {eval_file}")

    except Exception as e:
        logger.error(f"Evaluation failed: {e}")
        eval_results = {}

    # Step 6: Summary report
    logger.info("\n" + "="*80)
    logger.info("FINAL SUMMARY REPORT")
    logger.info("="*80)

    print("\nLoad Test Results by Concurrency:")
    print(f"{'Concurrency':<12} {'Throughput':<18} {'TTFT P99':<12} {'TPOT P95':<12} {'E2E P99':<12} {'SLO Compl':<10}")
    print("-" * 80)

    for summary in all_metrics_summary:
        print(f"{summary['concurrency']:<12} "
              f"{summary['throughput_tokens_per_sec']:<18.1f} "
              f"{summary['ttft_p99_ms']:<12.1f} "
              f"{summary['tpot_p95_ms']:<12.1f} "
              f"{summary['e2e_p99_ms']:<12.1f} "
              f"{summary['slo_compliance']*100:<10.1f}%")

    print("\nAccuracy Results:")
    if eval_results:
        for dataset_type in sorted(eval_results.keys()):
            if dataset_type != "overall":
                result = eval_results[dataset_type]
                print(f"  {dataset_type.upper():<10s}: {result['em']*100:6.2f}% "
                      f"({result['correct_count']}/{result['total_count']})")

        overall = eval_results.get("overall", {})
        print(f"  {'OVERALL':<10s}: {overall.get('em', 0)*100:6.2f}% "
              f"({overall.get('correct_count', 0)}/{overall.get('total_count', 0)})")
    else:
        print("  No evaluation results available")

    # Save summary
    summary_file = os.path.join(output_dir, "summary.json")
    with open(summary_file, 'w') as f:
        json.dump({
            "load_test_summary": all_metrics_summary,
            "eval_results": eval_results,
            "config": {
                "model_name": model_name,
                "variant": "med",
                "num_requests": num_requests,
                "concurrencies": concurrencies,
                "device": device
            }
        }, f, indent=2)

    logger.info("\n" + "="*80)
    logger.info("BASELINE EVALUATION COMPLETE")
    logger.info(f"Results saved to: {output_dir}")
    logger.info("="*80)
    logger.info("\nFiles generated:")
    logger.info(f"  - summary.json (overall summary)")
    logger.info(f"  - eval_results.json (accuracy metrics)")
    for concurrency in concurrencies:
        logger.info(f"  - metrics_concurrency_{concurrency}.json")
        logger.info(f"  - requests_concurrency_{concurrency}.jsonl")

    # Optionally return things for further notebook analysis
    return {
        "load_test_summary": all_metrics_summary,
        "eval_results": eval_results,
        "config": config,
    }


In [ ]:
results = run_pipeline()

2026-01-15 00:51:37,312 - __main__ - INFO - ================================================================================
2026-01-15 00:51:37,312 - __main__ - INFO - END-TO-END BASELINE EVALUATION: MED-ONLY SERVER (8-BIT QUANTIZATION)
2026-01-15 00:51:37,313 - __main__ - INFO - ================================================================================
2026-01-15 00:51:37,313 - __main__ - INFO - 
[STEP 0] PREPROCESSING DATASETS
2026-01-15 00:51:37,313 - __main__ - INFO - --------------------------------------------------------------------------------
2026-01-15 00:51:37,389 - preprocessing - INFO - Initialized DataPreprocessor
2026-01-15 00:51:37,389 - preprocessing - INFO -   Data dir: ./results/baseline_med
2026-01-15 00:51:37,390 - preprocessing - INFO -   Output dir: ./data/processed
2026-01-15 00:51:37,390 - preprocessing - INFO - ======================================================================
2026-01-15 00:51:37,390 - preprocessing - INFO - STARTING DATA PREPROCESS

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2026-01-15 00:51:57,140 - server - INFO - Model loaded successfully
2026-01-15 00:51:57,142 - server - INFO - Model size: 6.74B parameters
2026-01-15 00:51:57,142 - __main__ - INFO - 
[STEP 3] RUNNING LOAD TESTS
2026-01-15 00:51:57,143 - __main__ - INFO - --------------------------------------------------------------------------------
2026-01-15 00:51:57,143 - __main__ - INFO - 
>>> Testing with concurrency=1
2026-01-15 00:51:57,144 - load_generator - INFO - Initialized ClosedLoopLoadGenerator
2026-01-15 00:51:57,144 - load_generator - INFO -   Concurrency: 1
2026-01-15 00:51:57,145 - load_generator - INFO -   Total requests: 5000
2026-01-15 00:51:57,145 - load_generator - INFO -   Data pool size: 4873
2026-01-15 00:51:57,146 - load_generator - INFO - ======================================================================
2026-01-15 00:51:57,146 - load_generator - INFO - STARTING LOAD TEST: 5000 requests @ 1 concurrency
2026-01-15 00:51:57,146 - load_generator - INFO - =================


LOAD TEST RESULTS (Concurrency 1)

SUMMARY:
  Total Requests:             2
  Successful:                 2
  Failed:                     0
  Success Rate:          100.00%
  Total Duration:          9.85 seconds
  Throughput:               0.2 tokens/sec
  SLO Compliance:          0.00%
  SLO Violations:             2
  Escalation Rate:         0.00%

TTFT (Time-to-First-Token) in milliseconds:
  P50:  3753.86 ms
  P75:  4892.87 ms
  P90:  5576.28 ms
  P95:  5804.08 ms
  P99:  5986.33 ms
  Mean: 3753.86 ms (±2278.03)

TPOT (Time-Per-Output-Token) in milliseconds:
  P50:  1167.85 ms
  P75:  1220.17 ms
  P90:  1251.56 ms
  P95:  1262.02 ms
  P99:  1270.39 ms
  Mean: 1167.85 ms (±104.63)

E2E Latency (End-to-End) in milliseconds:
  P50:  4924.49 ms
  P75:  6116.71 ms
  P90:  6832.04 ms
  P95:  7070.48 ms
  P99:  7261.24 ms
  Mean: 4924.49 ms (±2384.43)

Queue Wait Time in milliseconds:
  P50:     0.02 ms
  P95:     0.03 ms
  P99:     0.03 ms
  Mean:    0.02 ms

Sample SLO Violations (fi

2026-01-15 00:52:19,426 - load_generator - INFO - Load test complete in 12.4s
2026-01-15 00:52:19,426 - metrics - INFO - Initialized MetricsCalculator with 4 metrics
2026-01-15 00:52:19,430 - metrics - INFO - Saved metrics to ./data/processed/metrics_concurrency_2.json
2026-01-15 00:52:19,430 - load_generator - INFO - Saved metrics to ./data/processed/requests_concurrency_2.jsonl
2026-01-15 00:52:19,430 - __main__ - INFO - 
>>> Testing with concurrency=4
2026-01-15 00:52:19,431 - load_generator - INFO - Initialized ClosedLoopLoadGenerator
2026-01-15 00:52:19,431 - load_generator - INFO -   Concurrency: 4
2026-01-15 00:52:19,431 - load_generator - INFO -   Total requests: 5000
2026-01-15 00:52:19,431 - load_generator - INFO -   Data pool size: 4873
2026-01-15 00:52:19,431 - load_generator - INFO - ======================================================================
2026-01-15 00:52:19,432 - load_generator - INFO - STARTING LOAD TEST: 5000 requests @ 4 concurrency
2026-01-15 00:52:19,4


LOAD TEST RESULTS (Concurrency 2)

SUMMARY:
  Total Requests:             4
  Successful:                 4
  Failed:                     0
  Success Rate:          100.00%
  Total Duration:         12.42 seconds
  Throughput:               0.3 tokens/sec
  SLO Compliance:          0.00%
  SLO Violations:             4
  Escalation Rate:         0.00%

TTFT (Time-to-First-Token) in milliseconds:
  P50:  3233.97 ms
  P75:  3989.69 ms
  P90:  4286.63 ms
  P95:  4385.60 ms
  P99:  4464.79 ms
  Mean: 3360.24 ms (±829.76)

TPOT (Time-Per-Output-Token) in milliseconds:
  P50:  2885.73 ms
  P75:  3065.75 ms
  P90:  3143.40 ms
  P95:  3169.28 ms
  P99:  3189.98 ms
  Mean: 2815.01 ms (±340.60)

E2E Latency (End-to-End) in milliseconds:
  P50:  6209.28 ms
  P75:  6808.61 ms
  P90:  7229.42 ms
  P95:  7369.69 ms
  P99:  7481.90 ms
  Mean: 6179.55 ms (±996.33)

Queue Wait Time in milliseconds:
  P50:     0.00 ms
  P95:     0.01 ms
  P99:     0.01 ms
  Mean:    0.00 ms

Sample SLO Violations (firs

2026-01-15 00:52:45,596 - load_generator - INFO - Load test complete in 26.2s
2026-01-15 00:52:45,597 - metrics - INFO - Initialized MetricsCalculator with 8 metrics
2026-01-15 00:52:45,600 - metrics - INFO - Saved metrics to ./data/processed/metrics_concurrency_4.json
2026-01-15 00:52:45,601 - load_generator - INFO - Saved metrics to ./data/processed/requests_concurrency_4.jsonl
2026-01-15 00:52:45,601 - __main__ - INFO - 
>>> Testing with concurrency=8
2026-01-15 00:52:45,601 - load_generator - INFO - Initialized ClosedLoopLoadGenerator
2026-01-15 00:52:45,602 - load_generator - INFO -   Concurrency: 8
2026-01-15 00:52:45,602 - load_generator - INFO -   Total requests: 5000
2026-01-15 00:52:45,602 - load_generator - INFO -   Data pool size: 4873
2026-01-15 00:52:45,602 - load_generator - INFO - ======================================================================
2026-01-15 00:52:45,603 - load_generator - INFO - STARTING LOAD TEST: 5000 requests @ 8 concurrency
2026-01-15 00:52:45,6


LOAD TEST RESULTS (Concurrency 4)

SUMMARY:
  Total Requests:             8
  Successful:                 8
  Failed:                     0
  Success Rate:          100.00%
  Total Duration:         26.16 seconds
  Throughput:               0.3 tokens/sec
  SLO Compliance:          0.00%
  SLO Violations:             8
  Escalation Rate:         0.00%

TTFT (Time-to-First-Token) in milliseconds:
  P50:  6644.73 ms
  P75:  6733.25 ms
  P90:  6903.55 ms
  P95:  6928.60 ms
  P99:  6948.65 ms
  Mean: 6645.35 ms (±215.09)

TPOT (Time-Per-Output-Token) in milliseconds:
  P50:  6160.02 ms
  P75:  7010.10 ms
  P90:  7105.19 ms
  P95:  7121.94 ms
  P99:  7135.35 ms
  Mean: 6277.51 ms (±681.63)

E2E Latency (End-to-End) in milliseconds:
  P50:  12702.44 ms
  P75:  13640.99 ms
  P90:  13792.09 ms
  P95:  13808.04 ms
  P99:  13820.80 ms
  Mean: 12927.30 ms (±649.90)

Queue Wait Time in milliseconds:
  P50:     0.00 ms
  P95:     0.00 ms
  P99:     0.00 ms
  Mean:    0.00 ms

Sample SLO Violations

2026-01-15 00:53:42,967 - load_generator - INFO - Load test complete in 57.4s
2026-01-15 00:53:42,968 - metrics - INFO - Initialized MetricsCalculator with 16 metrics
2026-01-15 00:53:42,971 - metrics - INFO - Saved metrics to ./data/processed/metrics_concurrency_8.json
2026-01-15 00:53:42,972 - load_generator - INFO - Saved metrics to ./data/processed/requests_concurrency_8.jsonl
2026-01-15 00:53:42,973 - __main__ - INFO - 
>>> Testing with concurrency=16
2026-01-15 00:53:42,973 - load_generator - INFO - Initialized ClosedLoopLoadGenerator
2026-01-15 00:53:42,973 - load_generator - INFO -   Concurrency: 16
2026-01-15 00:53:42,973 - load_generator - INFO -   Total requests: 5000
2026-01-15 00:53:42,974 - load_generator - INFO -   Data pool size: 4873
2026-01-15 00:53:42,974 - load_generator - INFO - ======================================================================
2026-01-15 00:53:42,974 - load_generator - INFO - STARTING LOAD TEST: 5000 requests @ 16 concurrency
2026-01-15 00:53:


LOAD TEST RESULTS (Concurrency 8)

SUMMARY:
  Total Requests:            16
  Successful:                16
  Failed:                     0
  Success Rate:          100.00%
  Total Duration:         57.36 seconds
  Throughput:               0.3 tokens/sec
  SLO Compliance:          0.00%
  SLO Violations:            16
  Escalation Rate:         0.00%

TTFT (Time-to-First-Token) in milliseconds:
  P50:  14364.14 ms
  P75:  15574.28 ms
  P90:  16601.48 ms
  P95:  17008.76 ms
  P99:  17124.68 ms
  Mean: 14706.20 ms (±1293.61)

TPOT (Time-Per-Output-Token) in milliseconds:
  P50:  13347.26 ms
  P75:  15820.80 ms
  P90:  17352.11 ms
  P95:  17437.44 ms
  P99:  17597.75 ms
  Mean: 13707.72 ms (±2705.27)

E2E Latency (End-to-End) in milliseconds:
  P50:  28444.74 ms
  P75:  29468.61 ms
  P90:  31493.13 ms
  P95:  31576.07 ms
  P99:  31760.81 ms
  Mean: 28420.82 ms (±2088.88)

Queue Wait Time in milliseconds:
  P50:     0.00 ms
  P95:     0.01 ms
  P99:     0.01 ms
  Mean:    0.00 ms

Sample

In [ ]:

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(
#         description="End-to-end baseline MED-only server evaluation"
#     )
    
#     # Data configuration
#     parser.add_argument("--preprocess", action="store_true",
#                        help="Run preprocessing (download and process datasets)")
#     parser.add_argument("--data_dir", default="data/raw",
#                        help="Raw data directory")
#     parser.add_argument("--processed_dir", default="data/processed",
#                        help="Processed data directory")
#     parser.add_argument("--data_subset", type=int, default=0,
#                        help="Use subset of data (0=all, >0=limit to N examples)")
    
#     # Model configuration
#     parser.add_argument("--model_name", 
#                        default="meta-llama/Llama-2-7b-chat-hf",
#                        help="HuggingFace model name")
#     parser.add_argument("--device", default="cuda",
#                        help="Device: 'cuda' or 'cpu'")
#     parser.add_argument("--dtype", default="auto",
#                        help="Data type: 'auto', 'float16', 'bfloat16'")
    
#     # Load test configuration
#     parser.add_argument("--num_requests", type=int, default=5000,
#                        help="Number of requests per concurrency level")
#     parser.add_argument("--concurrencies", type=int, nargs="+",
#                        default=[1, 2, 4, 8, 16, 32],
#                        help="Concurrency levels to test")
    
#     # Output configuration
#     parser.add_argument("--output_dir", default="results/baseline_med",
#                        help="Output directory for results")
    
#     args = parser.parse_args()
    
#     logger.info(f"Configuration:")
#     logger.info(f"  Model: {args.model_name}")
#     logger.info(f"  Device: {args.device}")
#     logger.info(f"  Requests per test: {args.num_requests}")
#     logger.info(f"  Concurrency levels: {args.concurrencies}")
#     logger.info(f"  Output dir: {args.output_dir}")
    
#     main(args)
